**Red- light cameras: Track vehicle trajectory after detection --> determine if it crossed line during red**


In [ ]:
!pip install ultralytics opencv-python numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.1 MB/s eta 0:00:00


In [ ]:
from google.colab import files

uploaded = files.upload()


Saving redlight.mp4 to redlight (1).mp4


In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load YOLO model
model = YOLO("yolov8n.pt")

# Open uploaded video
video_path = "redlight.mp4"
cap = cv2.VideoCapture(video_path)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Output video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(
    "output.mp4",
    fourcc,
    fps,
    (frame_width, frame_height)
)

# Tracking variables
track_window = None
roi_hist = None
tracking_initialized = False

# Simulated red light
red_light = True

# Stop line position
line_y = 300

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Draw stop line
    cv2.line(frame, (0, line_y), (frame_width, line_y), (0, 0, 255), 3)

    # Detect vehicle (only once)
    if not tracking_initialized:

        results = model(frame)[0]

        for box in results.boxes:
            cls = int(box.cls[0])

            # Class 2 = car
            if cls == 2:
                x1, y1, x2, y2 = map(int, box.xyxy[0])

                track_window = (x1, y1, x2 - x1, y2 - y1)

                roi = frame[y1:y2, x1:x2]
                hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

                roi_hist = cv2.calcHist(
                    [hsv_roi], [0], None, [180], [0, 180]
                )
                cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

                tracking_initialized = True
                break

    # Mean Shift tracking
    if tracking_initialized:

        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        back_proj = cv2.calcBackProject(
            [hsv], [0], roi_hist, [0, 180], 1
        )

        _, track_window = cv2.meanShift(
            back_proj,
            track_window,
            (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1),
        )

        x, y, w, h = track_window

        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        center_y = y + h // 2

        if red_light and center_y > line_y:
            cv2.putText(
                frame,
                "RED LIGHT VIOLATION!",
                (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (0, 0, 255),
                3,
            )

    # Save frame
    out.write(frame)

# Release
cap.release()
out.release()

print(" Done! Output saved as output.mp4")



0: 640x384 4 cars, 1 truck, 287.1ms
Speed: 6.9ms preprocess, 287.1ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 384)
✅ Done! Output saved as output.mp4


In [ ]:
from google.colab import files
files.download("output.mp4")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Multi-vehicle tracking + red light violation

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving redlight.mp4 to redlight (2).mp4


In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load YOLO model
model = YOLO("yolov8n.pt")

# Open video
cap = cv2.VideoCapture("redlight.mp4")

# Video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("output.mp4", fourcc, fps, (frame_width, frame_height))

# Simulated red light
red_light = True

# Stop line position
line_y = 300

# Store vehicle positions to detect crossing
vehicle_last_positions = {}

# Track violations
violated_ids = set()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Draw stop line
    cv2.line(frame, (0, line_y), (frame_width, line_y), (0, 0, 255), 3)

    # YOLO tracking (multi-object)
    results = model.track(frame, persist=True)[0]

    if results.boxes is not None and results.boxes.id is not None:

        boxes = results.boxes.xyxy.cpu().numpy()
        ids = results.boxes.id.cpu().numpy()
        classes = results.boxes.cls.cpu().numpy()

        for box, track_id, cls in zip(boxes, ids, classes):

            # Only track vehicles (car=2, truck=7, bus=5, motorcycle=3)
            if int(cls) not in [2, 3, 5, 7]:
                continue

            x1, y1, x2, y2 = map(int, box)
            center_y = (y1 + y2) // 2

            # Draw bounding box + ID
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(
                frame,
                f"ID {int(track_id)}",
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (255, 255, 0),
                2,
            )

            # Check crossing logic
            if track_id in vehicle_last_positions:
                prev_y = vehicle_last_positions[track_id]

                # Detect crossing from above to below line
                if prev_y < line_y and center_y >= line_y:

                    if red_light:
                        violated_ids.add(track_id)

            vehicle_last_positions[track_id] = center_y

            # Show violation label
            if track_id in violated_ids:
                cv2.putText(
                    frame,
                    "VIOLATION",
                    (x1, y2 + 25),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.8,
                    (0, 0, 255),
                    2,
                )

    # Show total violations
    cv2.putText(
        frame,
        f"Violations: {len(violated_ids)}",
        (20, 50),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 0, 255),
        3,
    )

    # Save frame
    out.write(frame)

cap.release()
out.release()

print(" Done! Output saved as output.mp4")


requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 2 packages in 250ms
Prepared 1 package in 81ms
Installed 1 package in 2ms
 + lap==0.5.12

requirements: AutoUpdate success ✅ 0.9s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


0: 640x384 4 cars, 1 truck, 171.6ms
Speed: 6.0ms preprocess, 171.6ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 4 cars, 1 truck, 154.9ms
Speed: 3.4ms preprocess, 154.9ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 4 cars, 1 truck, 147.0ms
Speed: 3.7ms preprocess, 147.0ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 4 cars, 1 truck, 166.9ms
Speed: 4.0ms preprocess, 166.9ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 4 cars, 1 truck, 150.7ms
Speed: 3.3ms preprocess, 150.7ms inference, 1.2ms postprocess per 

In [ ]:
from google.colab import files
files.download("output.mp4")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>